In [56]:
import json
import numpy as np
import nltk
import os
import pandas as pd
import re

In [34]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Asalamatina/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [51]:
from math import log
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from time import time

In [2]:
!pip install pymystem3

In [57]:
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

In [58]:
k = 2.0
trained_size = 2000
N = trained_size

### Обработка датасета

In [59]:
def sort_list(arr):
    return [x[0] for x in sorted(enumerate(arr), key=lambda x:x[1], reverse=True)]

In [60]:
def lemmatize(text):
    return [morph.lemmatize(token)[0] for token in nltk.word_tokenize(text)]

In [61]:
questions = pd.read_csv('quora_question_pairs_rus.csv')

In [62]:
questions.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [63]:
train_texts = questions[:trained_size]['question2'].tolist()

In [64]:
with open('lemmatized.json', 'w') as f:
    f.write(json.dumps(train_texts))

In [65]:
with open('lemmatized.json', 'r') as f:
    train_texts = json.loads(f.read())[:trained_size]

### Text length

In [66]:
text_length = [len(text.split()) for text in train_texts]
avgdl = sum(text_length)/N
avgdl

9.5675

In [67]:
X = vectorizer.fit_transform(train_texts)
count_matrix = X.toarray()
count_matrix.shape

(2000, 6632)

### Tf Idf matrix

In [68]:
tf_matrix = count_matrix / np.array(text_length).reshape((-1, 1))
tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
vocabulary = vectorizer.get_feature_names()
vocabulary[900:910]

['болтунщиком',
 'боль',
 'больно',
 'большая',
 'больше',
 'большей',
 'большие',
 'большим',
 'большинстве',
 'большинство']

In [70]:
def preprocess(text, lemm=False):
    if lemm:
        words = lemmatize(text)
    else:
        words = nltk.word_tokenize(text)
    query_modified = list(set(words).intersection(set(vocabulary)))  
    return query_modified

In [71]:
in_n_docs = np.count_nonzero(count_matrix, axis=0)
in_n_docs

array([ 7, 14,  7, ...,  1,  1,  1])

In [72]:
def IDF_modified(word):
    word_id = vocabulary.index(word)
    n = in_n_docs[word_id]
    return log((N - n + 0.5) / (n + 0.5))

In [73]:
IDF_modified('воде')

7.195187320178709

In [74]:
idfs = [IDF_modified(word) for word in vocabulary]
idfs[1000:1010]

[6.6838614462772235,
 7.195187320178709,
 5.893901832250363,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709,
 7.195187320178709]

In [77]:
def modify_tf(tf_value, doc_index, b=0.75):
    l = text_length[doc_index]
    return (tf_value * (k + 1.0))/(tf_value + k * (1.0 - b + b * (l/avgdl)))

def modify_tf_matrix(tf_matrix, b=0.75): 
    enumed =  np.ndenumerate(tf_matrix)
    for i, tf_value in enumed:
        doc_index = i[0]
        tf_matrix[i] = modify_tf(tf_value, doc_index, b)
    return tf_matrix

In [78]:

modified_tf_matrix = modify_tf_matrix(tf_matrix)
modified_tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Функция ранжирования bm25

## Задача 1

In [90]:
query = ' критический порог уже пройден'

In [91]:
def bm25_vector(query, lemm=False):
    vector = np.array(vectorizer.transform([' '.join(preprocess(query, lemm))]).todense())[0]
    binary_vector = np.vectorize(lambda x: 1.0 if x != 0.0 else 0.0)(vector) 
    idfs_from_query = np.array(idfs)*np.array(binary_vector)
    return modified_tf_matrix.dot(idfs_from_query) 

In [92]:

def bm25_iter(query, lemm=False):
    query_words = preprocess(query, lemm)
    relevance = []
    for i in range(N):
        doc_index = i
        doc_bm25 = 0.0
        for word in set(query_words): 
            word_index = vocabulary.index(word)
            tf_value = tf_matrix[(doc_index, word_index)]
            doc_bm25 += idfs[word_index] * modify_tf(tf_value, doc_index)
        relevance.append(doc_bm25)
    return relevance

In [95]:
start = time()
print('time for non-lemmatized query: ' + str(time() - start))
start = time()
print('time for lemmatized query: ' + str(time() - start))

time for non-lemmatized query: 3.600120544433594e-05
time for lemmatized query: 3.218650817871094e-05


In [96]:
start = time()
print('TIME non-lemmatized query:', str(time() - start))
start = time()
print('TIME lemmatized query:', str(time() - start))

TIME non-lemmatized query: 4.506111145019531e-05
TIME lemmatized query: 3.910064697265625e-05


### Задача 2

In [83]:
def search(query, lemm=False, n=N, nonzero=False, vector=True): 
    '''
    searches a given query, returns top n results, by default n = all found (the legth of collection)
    vector flag defines the algorythm (vector is used by default) 
    return format: [(document rank, document id, document text, bm_25), ...]
    '''
    
    if vector:
        bms = bm25_vector(query, lemm)
    else:
        bms = bm25_iter(query, lemm)
    relevance_sorted_document_ids_top_n = sort_list(bms)[:n]
    return [(rank, index, np.array(train_texts)[index], bms[index]) for rank, index in enumerate(relevance_sorted_document_ids_top_n)]

In [89]:
query = 'ребенок плачет в самолете уже два часа'
print('query:', query, '\n')
start = time()
response = search(query, n=10, lemm=True)
for rank, document_index, text, bm_25 in response:
    print('relevance rank:', rank+1)
    print('document:', text)
    print('bm 25 =', bm_25, '\n')


query: ребенок плачет в самолете уже два часа 

relevance rank: 1
document: у знаменитостей Болливуда есть свой собственный реактивный самолет
bm 25 = 1.4357859672020814 

relevance rank: 2
document: вы когда-нибудь сожалеете о том, что ваш ребенок
bm 25 = 1.3337518599932008 

relevance rank: 3
document: была инициатива по контролю за оружием, чтобы забрать оружие, которое уже есть у людей
bm 25 = 0.5177404144269768 

relevance rank: 4
document: что является лучшим оправданием для объяснения пробела в один или два года в вашей карьере работы для собеседования
bm 25 = 0.3298122373520815 

relevance rank: 5
document: сколько способов 12 яблок могут быть распределены среди 4 детей, так что каждый ребенок получает по меньшей мере 2 яблока
bm 25 = 0.2720249178995481 

relevance rank: 6
document: болливудская индустрия не поощряет аутсайдеров для экс-аутсайдеров, может получить одну или максимум два шанса, в то время как у звездных детей есть много шансов на выполнение
bm 25 = 0.167119098181

In [40]:
testable = questions[(questions['is_duplicate'] == 1)&(questions.index < trained_size)][:100]
testable.head()

,Unnamed: 0,question1,question2,is_duplicate
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1
6,6,как я могу быть хорошим геологом?,"что я должен делать, чтобы быть великим геологом?",1
10,10,как мне читать и находить комментарии к YouTube,как я могу увидеть все мои комментарии к YouTube,1
11,11,что может сделать физику легкой для изучения,как вы можете легко научиться физике,1
12,12,"какой был ваш первый сексуальный опыт, как",какой был ваш первый сексуальный опыт,1


In [41]:
testable.index

Int64Index([  4,   6,  10,  11,  12,  14,  15,  17,  19,  28,  30,  31,  37,
             47,  48,  49,  50,  52,  57,  61,  64,  65,  66,  70,  71,  72,
             73,  78,  83,  84,  85,  87,  91,  92,  94,  99, 103, 106, 112,
            119, 121, 124, 126, 134, 135, 142, 143, 151, 155, 157, 158, 159,
            162, 164, 167, 172, 174, 175, 177, 178, 179, 181, 184, 187, 188,
            189, 190, 192, 193, 196, 197, 198, 199, 202, 208, 209, 214, 215,
            218, 219, 220, 223, 225, 228, 234, 235, 237, 241, 242, 243, 245,
            248, 249, 250, 252, 254, 259, 260, 261, 266],
           dtype='int64')

In [42]:
search(questions.iloc[4]['question1'], n=5)

[(0, 1432, 'мне все равно, что люди думают обо мне', 2.8942597405799324),
 (1, 1761, 'что такое свет из', 2.5114209976934676),
 (2, 494, 'что это за картина?', 2.4201115560053226),
 (3, 59, 'это надежные торренты', 2.304579802827789),
 (4, 1669, 'как это влюбиться', 2.304579802827789)]

In [43]:
def test_q1_by_id(q1_index):
    top_5_ids = [i for rank, i, text, bm_25 in search(questions.iloc[q1_index]['question1'], n=5)]
    return 1.0 if q1_index in top_5_ids else 0.0

In [44]:
test_q1_by_id(6)

1.0

In [45]:
def test_rank_of_q1_by_id(q1_index):
    top_5_ids = [i for rank, i, text, bm_25 in search(questions.iloc[q1_index]['question1'], n=5)]
    if q1_index in top_5_ids:
        return 1/(top_5_ids.index(q1_index)+1)
    else:
        return 0.0

In [46]:
test_rank_of_q1_by_id(6)

0.3333333333333333

In [47]:
def test_over_multiple_questions(b, ranked=False, testsize=100): # test for different bs
    testable = questions[(questions['is_duplicate'] == 1)&(questions.index < trained_size)]
    if testsize < len(testable):
        testable = testable[:testsize]
        print('testing on', testsize, 'questions')
    else:
        print('testing on', len(testable), 'questions')
    modified_tf_matrix = modify_tf_matrix(tf_matrix, b=b)
    if ranked:
        test_q1 = test_rank_of_q1_by_id
    else:
        test_q1 = test_q1_by_id
    hit_count = 0.0
    for index in testable.index:
        hit_count += test_q1(index)
    return hit_count/len(testable.index)

In [48]:
bs = {'BM25': 0.75, 'BM15': 0, 'BM11': 1}
for key in bs:
    b = bs[key]
    print(key, 'b =', b)
    print('boolean precision:', test_over_multiple_questions(b, testsize=100))
    print('rank precision:', test_over_multiple_questions(b, ranked=True, testsize=100))
    print('')

BM25 b = 0.75
testing on 100 questions
boolean precision: 0.51
testing on 100 questions
rank precision: 0.36166666666666664

BM15 b = 0
testing on 100 questions
boolean precision: 0.55
testing on 100 questions
rank precision: 0.5126666666666667

BM11 b = 1
testing on 100 questions
boolean precision: 0.57
testing on 100 questions
rank precision: 0.4620000000000002



In [49]:
bs = {'BM25': 0.75, 'BM15': 0, 'BM11': 1}
for key in bs:
    b = bs[key]
    print(key, 'b =', b)
    print('boolean precision:', test_over_multiple_questions(b, testsize=500))
    print('rank precision:', test_over_multiple_questions(b, ranked=True, testsize=500))
    print('')

BM25 b = 0.75
testing on 500 questions
boolean precision: 0.61
testing on 500 questions
rank precision: 0.5240333333333336

BM15 b = 0
testing on 500 questions
boolean precision: 0.668
testing on 500 questions
rank precision: 0.5833666666666667

BM11 b = 1
testing on 500 questions
boolean precision: 0.664
testing on 500 questions
rank precision: 0.5447000000000001

